In [95]:
import numpy as np
from amplpy import AMPL

class WaterNetworkModel:
    def __init__(self, model_path, data_dir, data_list, data_number):
        self.model_path = model_path
        self.data_dir = data_dir
        self.data_list = data_list
        self.delta = 0.1
        self.p = 1.852
        self.omega = 10.67
        self.ampl = AMPL()
        self.data_number = data_number
        self.load_model()
        self.extract_sets_and_parameters()

    def load_model(self):
        input_data_file = f"{self.data_dir}/{self.data_list[self.data_number]}.dat"
        print("Water Network:", self.data_list[self.data_number], "\n")
        self.ampl.read(self.model_path)
        self.ampl.readData(input_data_file)

    def extract_sets_and_parameters(self):
        # Extract sets
        self.nodes = set(self.ampl.getSet('nodes').to_list())
        self.pipes = set(self.ampl.getSet('pipes'))
        self.arcs = self.ampl.getSet('arcs')
        self.source = set(self.ampl.getSet('Source').to_list())

        # Extract parameters
        self.L = self.ampl.getParameter('L').to_dict()
        self.D = self.ampl.getParameter('D').to_dict()
        self.C = self.ampl.getParameter('C').to_dict()
        self.P = self.ampl.getParameter('P').to_dict()
        self.R = self.ampl.getParameter('R').to_dict()
        self.E = self.ampl.getParameter('E').to_dict()
        self.d = self.ampl.getParameter('d').to_dict()

        # Extract variables
        self.l = {index: 0 for index in self.ampl.get_variable("l").to_dict().keys()}
        self.q = {index: 0 for index in self.ampl.get_variable("q").to_dict().keys()}
        self.h = {index: 0 for index in self.ampl.get_variable("h").to_dict().keys()}

        self.indices_l = list(self.l.keys())
        self.indices_q = list(self.q.keys())
        self.indices_h = list(self.h.keys())

    def objective_flat(self, x):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        return sum(l[i, j, k] * self.C[k] for (i, j) in self.arcs for k in self.pipes)

    def flow_conservation_flat(self, x):
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        constraints = []
        for j in self.nodes:
            inflow = sum(q[i, j] for i in self.nodes if (i, j) in self.arcs)
            outflow = sum(q[j, i] for i in self.nodes if (j, i) in self.arcs)
            constraints.append(inflow - outflow - self.D[j])
        return constraints

    def head_loss_flat(self, x):
        delta = self.delta
        p = self.p
        a = (15 * (delta ** (p - 1)) / 8 +
                  ((p - 1) * p * delta ** (p - 1)) / 8 -
                  7 * p * (delta ** (p - 1)) / 8)
        b = (-5 * (delta ** (p - 3)) / 4 -
                  ((p - 1) * p * delta ** (p - 3)) / 4 +
                  5 * p * (delta ** (p - 3)) / 4)
        c = (3 * (delta ** (p - 5)) / 8 +
                  ((p - 1) * p * delta ** (p - 5)) / 8 -
                  3 * p * (delta ** (p - 5)) / 8)

        l_dict = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q_dict = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h_dict = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) +k] for k in range(len(self.indices_h))}
        
        constraints = []
        for (i, j) in self.arcs:
            # if -delta <= q_dict[i, j] <= delta:
            #     head_loss_term = (
            #         (0.001 ** 1.852) *
            #         (c * (q_dict[i, j] ** 5) + b * (q_dict[i, j] ** 3) + a * q_dict[i, j]) *
            #         sum(self.omega * l_dict[i, j, k] / (self.R[k] ** 1.852 * (self.d[k] / 1000) ** 4.87) for k in self.pipes)
            #     )
            # else:
            head_loss_term = (
                (q_dict[i, j] * abs(q_dict[i, j]) ** 0.852) *
                (0.001 ** 1.852) *
                sum(self.omega * l_dict[i, j, k] / (self.R[k] ** 1.852 * (self.d[k] / 1000) ** 4.87) for k in self.pipes)
            )
            constraints.append(h_dict[i] - h_dict[j] - head_loss_term)
        return constraints

    def pipe_length_constraint_flat(self, x):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        constraints = []
        for (i, j) in self.arcs:
            constraints.append(sum(l[i, j, k] for k in self.pipes) - self.L[i, j])
        return constraints

    def source_min_pressure_constraint_flat(self, x):
        h_dict = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        constraints = []
        for i in self.source:
            constraints.append(h_dict[i] - self.E[i])
        return constraints

    def pressure_constraints_flat(self, x):
        h_dict = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        constraints = []
        for i in self.nodes:
            if i not in self.source:
                constraints.append(h_dict[i] - (self.E[i] + self.P[i]))
        return constraints

    def length_constraint_flat(self, x):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        constraints = []
        for (i, j) in self.arcs:
            for k in self.pipes:
                constraints.append(-l[i, j, k])
        return constraints

    def calculate_constraints(self,x):
        constraints = []
        # Flow conservation constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.flow_conservation_flat(x)
        })
        # Head loss constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.head_loss_flat(x)
        })
        # Pipe length constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.pipe_length_constraint_flat(x)
        })
        # Source minimum pressure constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.source_min_pressure_constraint_flat(x)
        })
        # Pressure constraints (inequality)
        constraints.append({
            'type': 'ineq',
            'fun': lambda x: self.pressure_constraints_flat(x)
        })
        constraints.append({
            'type': 'ineq',
            'fun': lambda x: self.length_constraint_flat(x)
        })
        return constraints

    def lagrangian_multipliers(self, lambda0):
        lag_flow = {}
        lag_headloss = {}
        lag_length = {}
        lag_head_source = {}
        lag_head_non_source = {}
        lag_calculate_constraints = {}
        lag_length_constraint_flat = {}

        for idx, key in enumerate(list(self.nodes)):
            lag_flow[key] = lambda0[idx]
        for idx, key in enumerate(list(self.arcs)):
            lag_headloss[key] = lambda0[len(list(self.nodes))+idx]
        for idx, key in enumerate(list(self.arcs)):
            lag_length[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) +idx]
        for idx, key in enumerate(list(self.nodes)):
            if key == list(self.source)[0]:
                lag_head_source[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) + len(list(self.arcs)) +idx]
            else:
                lag_head_non_source[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) + len(list(self.arcs)) + idx]
        for idx, key in enumerate(self.indices_l):
            lag_length_constraint_flat[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) + len(list(self.arcs)) + len(list(self.nodes)) + idx]
            
        return lag_flow, lag_headloss, lag_length, lag_head_source, lag_head_non_source, lag_length_constraint_flat
               
    def del_L(self, l, q, h, lambda0):
        lag_flow, lag_headloss, lag_length, lag_head_source, lag_head_non_source, lag_length_constraint_flat = self.lagrangian_multipliers(lambda0)
        del_l = {}
        for (i, j) in q.keys():
            for k in self.d.keys():
                del_l[i, j, k] = self.C[k] - (
                    - (q[i, j] * abs(q[i, j])**0.852) * (0.001**1.852)
                    * (10.67*lag_headloss[i,j] / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87))
                ) + lag_length[i,j]  

        del_q = {}
        for (i, j) in q.keys():
            del_q[i, j] = lag_flow[j] - lag_flow[i] - (
                 (1.852 * lag_headloss[i,j] * abs(q[i, j])**0.852) * (0.001**1.852)
                * sum(10.67 * l[i, j, k] / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87) for k in self.d.keys())
            )
        del_h = {}
        for j in h.keys():
            if j == list(self.source)[0]:
                del_h[j] = - sum(lag_headloss[i,j] for i in h.keys() if (i,j) in q.keys()) + sum(lag_headloss[j,i] for i in h.keys() if (j,i) in q.keys()) - lag_head_source[j]
            else:
                del_h[j] = - sum(lag_headloss[i,j] for i in h.keys() if (i,j) in q.keys()) + sum(lag_headloss[j,i] for i in h.keys() if (j,i) in q.keys()) - lag_head_non_source[j]
        return del_l, del_q, del_h
    
    def lagrangian_gradient(self,x,lambda0):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        
        del_l, del_q, del_h = self.del_L(l, q, h, lambda0)

        grad = np.array([0]*len(x), dtype=np.float64)
        for idx, key in enumerate(list(self.indices_l)):
            # print(del_l[key])
            grad[idx] = del_l[key]
        for idx, key in enumerate(list(self.indices_q)):
            grad[len(list(self.indices_l)) + idx] = del_q[key]
        for idx, key in enumerate(list(self.indices_h)):
            grad[len(list(self.indices_l)) + len(list(self.indices_q)) + idx] = del_h[key]
        return grad
        
    def del_g2_del_l(self, l, q):
        del_l = {}
        for (i, j) in q.keys():
            for k in self.d.keys():
                del_l[i, j, k] = (
                    - (q[i, j] * abs(q[i, j])**0.852) * (0.001**1.852)
                    * (10.67 / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87))
                )
        return del_l
        
    def del_g2_del_q(self, l, q):
        del_q = {}
        for (i, j) in q.keys():
            del_q[i, j] = (
                - (1.852 * abs(q[i, j])**0.852) * (0.001**1.852)
                * sum(10.67 * l[i, j, k] / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87) for k in self.d.keys())
            )
        return del_q
        
    def evaluate_jacobian(self, x):
        # Reconstruct variables from x
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        
        num_variables = len(x)
        
        # Initialize the Jacobian matrix
        J = np.zeros((self.num_con, num_variables))
        del_l = self.del_g2_del_l(l, q)
        del_q = self.del_g2_del_q(l, q)
        # print(del_q)
        row_idx = 0
        for c in self.constraints:
            constraint_function = c['fun']
            constraint_values = np.atleast_1d(constraint_function(x))  # Evaluate constraint function
            for i, value in enumerate(constraint_values):
                # print(i, value)
                for var_idx, key in enumerate((self.indices_l)):
                    if row_idx == 0:
                        J[row_idx + i, var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)):
                        J[row_idx + i, var_idx] = del_l[self.indices_l[var_idx]]
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)):
                        if (self.indices_l[var_idx][0],self.indices_l[var_idx][1]) == list(self.arcs)[i]:
                            J[row_idx + i, var_idx] = 1
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)) + len(self.constraints[2]['fun'](x)) + len(self.constraints[3]['fun'](x)) + len(self.constraints[4]['fun'](x)):
                        # print("self.indices_l[var_idx]",self.indices_l[var_idx])
                        if i == var_idx:
                            # print(i, var_idx)
                            # print(key)
                            J[row_idx + i, var_idx] = -1
                    else:
                        J[row_idx + i, var_idx] = 0
                    
                for var_idx in range(len(self.indices_q)):
                    if row_idx == 0:
                        if self.indices_q[var_idx][0] == list(self.nodes)[i]:
                            J[row_idx + i, len(self.indices_l) + var_idx] = -1
                        elif self.indices_q[var_idx][1] == list(self.nodes)[i]:
                            J[row_idx + i, len(self.indices_l) + var_idx] = 1
                        else:
                            J[row_idx + i, len(self.indices_l) + var_idx] = 0
                    
                    elif row_idx == len(self.constraints[0]['fun'](x)):
                        if self.indices_q[var_idx] == list(self.arcs)[i]:
                            J[row_idx + i, len(self.indices_l) + var_idx] = del_q[self.indices_q[var_idx]]
                        else:
                            J[row_idx + i, len(self.indices_l) + var_idx] = 0
                    else:
                        J[row_idx + i, len(self.indices_l) + var_idx] = 0
                     
                for var_idx in range(len(self.indices_h)):
                    if row_idx == 0:
                        J[row_idx + i, len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)):
                        if  self.indices_h[var_idx] == self.indices_q[i][0]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 1
                        elif self.indices_h[var_idx] == self.indices_q[i][1]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = -1
                        else:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)):
                        J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)) + len(self.constraints[2]['fun'](x)):
                        if self.indices_h[var_idx] == list(self.source)[0]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 1
                        else:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)) + len(self.constraints[2]['fun'](x)) + len(self.constraints[3]['fun'](x)):
                        if self.indices_h[var_idx] == list(self.nodes)[i+1]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = -1
                        else:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0 
                   
            row_idx += len(constraint_values)
        return J

    def KKT_rhs(self, x, lambda0):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        del_l = self.del_g2_del_l(l, q)
        del_q = self.del_g2_del_q(l, q)
        
        kkt_rhs = np.array([0]*(len(x)+ len(lambda0)), dtype=np.float64)

        for var_idx in range(len(self.indices_l)):
            x_index = list(self.arcs).index((self.indices_l[var_idx][0],self.indices_l[var_idx][1]))
            kkt_rhs[var_idx] = -self.C[self.indices_l[var_idx][2]] - del_l[self.indices_l[var_idx]]* lambda0[len(list(self.nodes)) + x_index] - lambda0[len(list(self.nodes)) + len(list(self.arcs)) + x_index] + 1
        for var_idx in range(len(self.indices_q)):
            x_index = len(self.nodes) + list(self.arcs).index(self.indices_q[var_idx])
            kkt_rhs[len(self.indices_l) + var_idx] = -lambda0[list(self.nodes).index(self.indices_q[var_idx][0])] + lambda0[list(self.nodes).index(self.indices_q[var_idx][1])] - del_q[self.indices_q[var_idx]]*lambda0[x_index]
        for var_idx in range(len(self.indices_h)):
            u = self.indices_h[var_idx]
            if self.indices_h[var_idx] == list(self.source)[0]:
                i = self.indices_h[var_idx]
                kkt_rhs[len(self.indices_l) + len(self.indices_q) + var_idx] = -sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((i,j))] for j in self.nodes if (i, j) in self.arcs) + sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((j,i))] for j in self.nodes if (j, i) in self.arcs) - lambda0[len(list(self.nodes)) + 2*len(list(self.arcs)) + var_idx]
            else:
                i = self.indices_h[var_idx]
                kkt_rhs[len(self.indices_l) + len(self.indices_q) + var_idx] = -sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((i,j))] for j in self.nodes if (i, j) in self.arcs) + sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((j,i))] for j in self.nodes if (j, i) in self.arcs) + lambda0[len(list(self.nodes)) + 2*len(list(self.arcs)) + var_idx]

        for var_idx in range(len(self.indices_h)):
            kkt_rhs[len(x) + var_idx] = -self.constraints[0]['fun'](x)[var_idx]
        for var_idx in range(len(self.indices_q)):
            kkt_rhs[len(x) + len(self.indices_h) +  var_idx] = -self.constraints[1]['fun'](x)[var_idx]
        for var_idx in range(len(self.indices_q)):
            kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) +  var_idx] = -self.constraints[2]['fun'](x)[var_idx]
        for var_idx in range(len(self.indices_h)):
            if self.indices_h[var_idx] == list(self.source)[0]:
                kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) + len(self.indices_q) +  var_idx] = -self.constraints[3]['fun'](x)[var_idx]
            else:
                # print(self.constraints[4]['fun'](x)[var_idx])
                kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) + len(self.indices_q) +  var_idx] = -self.constraints[4]['fun'](x)[var_idx-1]   
        for var_idx in range(len(self.indices_l)):
            kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) + len(self.indices_q) + len(self.indices_h) +  var_idx] = -self.constraints[5]['fun'](x)[var_idx]
        return kkt_rhs
        
    def solve_kkt_newton(self, x, lambda0, hessian):
        J = self.evaluate_jacobian(x)
        # print("Jacobian Matrix",J)
        H = hessian
        
        KKT_matrix = np.block([
            [H, J.T],
            [J, np.zeros((J.shape[0], J.shape[0]))]
        ])
        # print(KKT_matrix)
        # KKT_rhs = np.hstack(self.KKT_rhs(x, lambda0))
        KKT_rhs = np.hstack([self.KKT_rhs(x, lambda0).reshape(-1, 1)])  # Ensure it's a column vector

        solution = np.linalg.solve(KKT_matrix, KKT_rhs)
        delta_x = solution[:len(x)]
        delta_lambda = solution[len(x):]
        return delta_x, delta_lambda
        
    def solve_qp_problem(self, x, lambda0, hessian):
        delta_x, delta_lambda = self.solve_kkt_newton(x, lambda0, hessian)
        
        # print(f"Optimal solution: {delta_x}")
        # print(f"Lagrange multipliers: {delta_lambda}")
        return delta_x, delta_lambda
        
    def line_search(self, x, delta_x, alpha=1.0, tol=1e-4):
        new_x = x + delta_x 
        
        penalty_old = self.penalty_function(x)
        penalty_new = self.penalty_function(new_x)
        print(penalty_old)
        print(penalty_old)
        while (penalty_new).all() >= (penalty_old).all() :
            alpha *= 0.5
            new_x = x + alpha * delta_x 
            penalty_new = self.penalty_function(new_x)
        
        return new_x, penalty_new
    
    def penalty_function(self, x):
        objective_val = self.objective_flat(x)
        violated_constraints = sum(
            sum(np.abs(c['fun'](x)[i]) for i in range(len(c['fun'](x))) if (np.abs(c['fun'](x)[i])).all() > 0)
            for c in self.constraints
        )
        # print("objective_val", objective_val)
        # print("violated_constraints:", violated_constraints)
        return objective_val + violated_constraints

    # def penalty_function(self, x):
    #     objective_val = self.objective_flat(x)
    #     violated_constraints = sum(
    #         np.sum(np.abs(c['fun'](x)) > 1e-6)  # Count the elements that violate the threshold
    #         for c in self.constraints
    #     )
    #     print(violated_constraints)
    #     return objective_val + violated_constraints
        
    def update_hessian(self, x_old, x_new, grad_old, grad_new):
        """
        Perform the BFGS Hessian update and update the stored Hessian matrix.
        
        Parameters:
            gamma_k (np.ndarray): Difference in gradients, γ^k = grad(x^{k+1}) - grad(x^k) (n-dimensional vector).
            delta_x_k (np.ndarray): Step vector, Δx^k = x^{k+1} - x^k (n-dimensional vector).
        """
        
        gamma_k = grad_new - grad_old
        delta_x_k = x_new - x_old
        
        # Current Hessian matrix
        H_k = self.hessian
        
        # Compute the scalar terms for the update
        gamma_k_T_delta_x_k = np.dot(gamma_k, delta_x_k)  # (γ^k)^T Δx^k
        delta_x_k_T_H_k_delta_x_k = np.dot(delta_x_k.T, np.dot(H_k, delta_x_k))  # (Δx^k)^T H^k Δx^k
        
        # Check the curvature condition for numerical stability
        # if gamma_k_T_delta_x_k <= 1e-8 or delta_x_k_T_H_k_delta_x_k <= 1e-8:
        #     print("Curvature condition not satisfied. Skipping update.")
        #     return
            
        # Compute the first term: (γ^k (γ^k)^T) / (γ^k)^T Δx^k
        term1 = np.outer(gamma_k, gamma_k) / gamma_k_T_delta_x_k
        
        # Compute the second term: (H^k Δx^k (Δx^k)^T H^k) / (Δx^k)^T H^k Δx^k
        H_k_delta_x_k = np.dot(H_k, delta_x_k)  # H^k Δx^k
        term2 = np.outer(H_k_delta_x_k, H_k_delta_x_k) / delta_x_k_T_H_k_delta_x_k
        
        # Update the Hessian matrix
        hessian = H_k + term1 - term2
        return hessian
        
    def convergence_check(self, x, new_x, tol=1e-6):
        return np.linalg.norm(new_x - x) < tol
        
    def sqpSolver(self):        
        max_Q = sum(self.D[k] for k in self.nodes if k not in self.source)
        # Initial guesses for decision variables
        x0_l = np.array([0] * len(self.indices_l))
        x0_q = np.array([max_Q] * len(self.indices_q))
        x0_h = np.array([210] * len(self.indices_h))
        x = np.concatenate([x0_l, x0_q, x0_h])
                
        iteration = 0
        tolerance = 1e-6
        print(f"x{iteration}:",x)
        
        self.hessian = np.eye(len(x))
        # print("self.hessian", self.hessian)
        self.constraints = self.calculate_constraints(x)
        
        self.num_con = 0
        for constraint in self.constraints:
            c_values = constraint['fun'](x)
            self.num_con += len(c_values) 
        print("Number of constraints:",self.num_con)
        
        lambda0 = np.zeros(self.num_con)
        
        delta_x, delta_lambda = self.solve_qp_problem(x, lambda0, self.hessian)
        x_new, penalty_new = self.line_search(x, delta_x)
        # print("x_new:", x_new)
        # print("delta_lambda:",delta_lambda)
        grad_old = self.lagrangian_gradient(x, delta_lambda)
        grad_new = self.lagrangian_gradient(x_new, delta_lambda)
        
        self.hessian = self.update_hessian(x, x_new, grad_old, grad_new)
        print(self.hessian)
        iteration += 1
        while not self.convergence_check(x, x_new, tolerance):
            x = x_new.copy()
            print(f"x{iteration}:",x_new)
            
            delta_x, delta_lambda = self.solve_qp_problem(x, delta_lambda, self.hessian)
            # print(delta_lambda)
            x_new, penalty_new = self.line_search(x, delta_x)
            
            grad_old = self.lagrangian_gradient(x, delta_lambda)
            grad_new = self.lagrangian_gradient(x_new, delta_lambda)
            
            self.hessian = self.update_hessian(x, x_new, grad_old, grad_new)
            # print(f"Hessian{iteration}:", self.hessian)
            
            iteration += 1
            print(self.convergence_check(x, x_new, tolerance))
            if iteration > 100:
                break
        print("x optimal:",x_new)
        print("optimal solution:", self.objective_flat(x_new))
        
model_path = '../m1Basic.mod'
data_dir = '/home/nitishdumoliya/waterNetwork/data'
data_list = [
    "d1_Sample_input_cycle_twoloop",
    "d2_Sample_input_cycle_hanoi",
]

import sys
# np.set_printoptions(threshold=sys.maxsize) # 1000
# np.set_printoptions(threshold=2000) # 1000

data_number = 0
water_network = WaterNetworkModel(model_path, data_dir, data_list, data_number)
water_network.sqpSolver()  

Water Network: d1_Sample_input_cycle_twoloop 

x0: [  0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.  

KeyboardInterrupt: 

In [3]:
import numpy as np
from amplpy import AMPL

class WaterNetworkModel:
    def __init__(self, model_path, data_dir, data_list, data_number):
        self.model_path = model_path
        self.data_dir = data_dir
        self.data_list = data_list
        self.delta = 0.1
        self.p = 1.852
        self.omega = 10.67
        self.ampl = AMPL()
        self.data_number = data_number
        self.load_model()
        self.extract_sets_and_parameters()

    def load_model(self):
        input_data_file = f"{self.data_dir}/{self.data_list[self.data_number]}.dat"
        print("Water Network:", self.data_list[self.data_number], "\n")
        self.ampl.read(self.model_path)
        self.ampl.readData(input_data_file)

    def extract_sets_and_parameters(self):
        # Extract sets
        self.nodes = set(self.ampl.getSet('nodes').to_list())
        self.pipes = set(self.ampl.getSet('pipes'))
        self.arcs = self.ampl.getSet('arcs')
        self.source = set(self.ampl.getSet('Source').to_list())

        # Extract parameters
        self.L = self.ampl.getParameter('L').to_dict()
        self.D = self.ampl.getParameter('D').to_dict()
        self.C = self.ampl.getParameter('C').to_dict()
        self.P = self.ampl.getParameter('P').to_dict()
        self.R = self.ampl.getParameter('R').to_dict()
        self.E = self.ampl.getParameter('E').to_dict()
        self.d = self.ampl.getParameter('d').to_dict()

        # Extract variables
        self.l = {index: 0 for index in self.ampl.get_variable("l").to_dict().keys()}
        self.q = {index: 0 for index in self.ampl.get_variable("q").to_dict().keys()}
        self.h = {index: 0 for index in self.ampl.get_variable("h").to_dict().keys()}

        self.indices_l = list(self.l.keys())
        self.indices_q = list(self.q.keys())
        self.indices_h = list(self.h.keys())

    def objective_flat(self, x):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        return sum(l[i, j, k] * self.C[k] for (i, j) in self.arcs for k in self.pipes)

    def flow_conservation_flat(self, x):
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        constraints = []
        for j in self.nodes:
            inflow = sum(q[i, j] for i in self.nodes if (i, j) in self.arcs)
            outflow = sum(q[j, i] for i in self.nodes if (j, i) in self.arcs)
            constraints.append(inflow - outflow - self.D[j])
        return constraints

    def head_loss_flat(self, x):
        delta = self.delta
        p = self.p
        a = (15 * (delta ** (p - 1)) / 8 +
                  ((p - 1) * p * delta ** (p - 1)) / 8 -
                  7 * p * (delta ** (p - 1)) / 8)
        b = (-5 * (delta ** (p - 3)) / 4 -
                  ((p - 1) * p * delta ** (p - 3)) / 4 +
                  5 * p * (delta ** (p - 3)) / 4)
        c = (3 * (delta ** (p - 5)) / 8 +
                  ((p - 1) * p * delta ** (p - 5)) / 8 -
                  3 * p * (delta ** (p - 5)) / 8)

        l_dict = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q_dict = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h_dict = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) +k] for k in range(len(self.indices_h))}
        
        constraints = []
        for (i, j) in self.arcs:
            # if -delta <= q_dict[i, j] <= delta:
            #     head_loss_term = (
            #         (0.001 ** 1.852) *
            #         (c * (q_dict[i, j] ** 5) + b * (q_dict[i, j] ** 3) + a * q_dict[i, j]) *
            #         sum(self.omega * l_dict[i, j, k] / (self.R[k] ** 1.852 * (self.d[k] / 1000) ** 4.87) for k in self.pipes)
            #     )
            # else:
            head_loss_term = (
                (q_dict[i, j] * abs(q_dict[i, j]) ** 0.852) *
                (0.001 ** 1.852) *
                sum(self.omega * l_dict[i, j, k] / (self.R[k] ** 1.852 * (self.d[k] / 1000) ** 4.87) for k in self.pipes)
            )
            constraints.append(h_dict[i] - h_dict[j] - head_loss_term)
        return constraints

    def pipe_length_constraint_flat(self, x):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        constraints = []
        for (i, j) in self.arcs:
            constraints.append(sum(l[i, j, k] for k in self.pipes) - self.L[i, j])
        return constraints

    def source_min_pressure_constraint_flat(self, x):
        h_dict = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        constraints = []
        for i in self.source:
            constraints.append(h_dict[i] - self.E[i])
        return constraints

    def pressure_constraints_flat(self, x):
        h_dict = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        constraints = []
        for i in self.nodes:
            if i not in self.source:
                constraints.append(h_dict[i] - (self.E[i] + self.P[i]))
        return constraints

    def length_constraint_flat(self, x):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        constraints = []
        for (i, j) in self.arcs:
            for k in self.pipes:
                constraints.append(-l[i, j, k])
        return constraints

    def calculate_constraints(self,x):
        constraints = []
        # Flow conservation constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.flow_conservation_flat(x)
        })
        # Head loss constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.head_loss_flat(x)
        })
        # Pipe length constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.pipe_length_constraint_flat(x)
        })
        # Source minimum pressure constraints
        constraints.append({
            'type': 'eq',
            'fun': lambda x: self.source_min_pressure_constraint_flat(x)
        })
        # Pressure constraints (inequality)
        constraints.append({
            'type': 'ineq',
            'fun': lambda x: self.pressure_constraints_flat(x)
        })
        constraints.append({
            'type': 'ineq',
            'fun': lambda x: self.length_constraint_flat(x)
        })
        return constraints

    def lagrangian_multipliers(self, lambda0):
        lag_flow = {}
        lag_headloss = {}
        lag_length = {}
        lag_head_source = {}
        lag_head_non_source = {}
        lag_calculate_constraints = {}
        lag_length_constraint_flat = {}

        for idx, key in enumerate(list(self.nodes)):
            lag_flow[key] = lambda0[idx]
        for idx, key in enumerate(list(self.arcs)):
            lag_headloss[key] = lambda0[len(list(self.nodes))+idx]
        for idx, key in enumerate(list(self.arcs)):
            lag_length[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) +idx]
        for idx, key in enumerate(list(self.nodes)):
            if key == list(self.source)[0]:
                lag_head_source[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) + len(list(self.arcs)) +idx]
            else:
                lag_head_non_source[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) + len(list(self.arcs)) + idx]
        for idx, key in enumerate(self.indices_l):
            lag_length_constraint_flat[key] = lambda0[len(list(self.nodes)) + len(list(self.arcs)) + len(list(self.arcs)) + len(list(self.nodes)) + idx]
            
        return lag_flow, lag_headloss, lag_length, lag_head_source, lag_head_non_source, lag_length_constraint_flat
               
    def del_L(self, l, q, h, lambda0):
        lag_flow, lag_headloss, lag_length, lag_head_source, lag_head_non_source, lag_length_constraint_flat = self.lagrangian_multipliers(lambda0)
        del_l = {}
        for (i, j) in q.keys():
            for k in self.d.keys():
                del_l[i, j, k] = self.C[k] - (
                    - (q[i, j] * abs(q[i, j])**0.852) * (0.001**1.852)
                    * (10.67*lag_headloss[i,j] / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87))
                ) + lag_length[i,j]  

        del_q = {}
        for (i, j) in q.keys():
            del_q[i, j] = lag_flow[j] - lag_flow[i] - (
                 (1.852 * lag_headloss[i,j] * abs(q[i, j])**0.852) * (0.001**1.852)
                * sum(10.67 * l[i, j, k] / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87) for k in self.d.keys())
            )
        del_h = {}
        for j in h.keys():
            if j == list(self.source)[0]:
                del_h[j] = - sum(lag_headloss[i,j] for i in h.keys() if (i,j) in q.keys()) + sum(lag_headloss[j,i] for i in h.keys() if (j,i) in q.keys()) - lag_head_source[j]
            else:
                del_h[j] = - sum(lag_headloss[i,j] for i in h.keys() if (i,j) in q.keys()) + sum(lag_headloss[j,i] for i in h.keys() if (j,i) in q.keys()) - lag_head_non_source[j]
        return del_l, del_q, del_h
    
    def lagrangian_gradient(self,x,lambda0):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        
        del_l, del_q, del_h = self.del_L(l, q, h, lambda0)

        grad = np.array([0]*len(x), dtype=np.float64)
        for idx, key in enumerate(list(self.indices_l)):
            # print(del_l[key])
            grad[idx] = del_l[key]
        for idx, key in enumerate(list(self.indices_q)):
            grad[len(list(self.indices_l)) + idx] = del_q[key]
        for idx, key in enumerate(list(self.indices_h)):
            grad[len(list(self.indices_l)) + len(list(self.indices_q)) + idx] = del_h[key]
        return grad
        
    def del_g2_del_l(self, l, q):
        del_l = {}
        for (i, j) in q.keys():
            for k in self.d.keys():
                del_l[i, j, k] = (
                    - (q[i, j] * abs(q[i, j])**0.852) * (0.001**1.852)
                    * (10.67 / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87))
                )
        return del_l
        
    def del_g2_del_q(self, l, q):
        del_q = {}
        for (i, j) in q.keys():
            del_q[i, j] = (
                - (1.852 * abs(q[i, j])**0.852) * (0.001**1.852)
                * sum(10.67 * l[i, j, k] / ((self.R[k]**1.852) * (self.d[k] / 1000)**4.87) for k in self.d.keys())
            )
        return del_q
        
    def evaluate_jacobian(self, x):
        # Reconstruct variables from x
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        
        num_variables = len(x)
        
        # Initialize the Jacobian matrix
        J = np.zeros((self.num_con, num_variables))
        del_l = self.del_g2_del_l(l, q)
        del_q = self.del_g2_del_q(l, q)
        # print(del_q)
        row_idx = 0
        for c in self.constraints:
            constraint_function = c['fun']
            constraint_values = np.atleast_1d(constraint_function(x))  # Evaluate constraint function
            for i, value in enumerate(constraint_values):
                # print(i, value)
                for var_idx, key in enumerate((self.indices_l)):
                    if row_idx == 0:
                        J[row_idx + i, var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)):
                        J[row_idx + i, var_idx] = del_l[self.indices_l[var_idx]]
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)):
                        if (self.indices_l[var_idx][0],self.indices_l[var_idx][1]) == list(self.arcs)[i]:
                            J[row_idx + i, var_idx] = 1
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)) + len(self.constraints[2]['fun'](x)) + len(self.constraints[3]['fun'](x)) + len(self.constraints[4]['fun'](x)):
                        # print("self.indices_l[var_idx]",self.indices_l[var_idx])
                        if i == var_idx:
                            # print(i, var_idx)
                            # print(key)
                            J[row_idx + i, var_idx] = -1
                    else:
                        J[row_idx + i, var_idx] = 0
                    
                for var_idx in range(len(self.indices_q)):
                    if row_idx == 0:
                        if self.indices_q[var_idx][0] == list(self.nodes)[i]:
                            J[row_idx + i, len(self.indices_l) + var_idx] = -1
                        elif self.indices_q[var_idx][1] == list(self.nodes)[i]:
                            J[row_idx + i, len(self.indices_l) + var_idx] = 1
                        else:
                            J[row_idx + i, len(self.indices_l) + var_idx] = 0
                    
                    elif row_idx == len(self.constraints[0]['fun'](x)):
                        if self.indices_q[var_idx] == list(self.arcs)[i]:
                            J[row_idx + i, len(self.indices_l) + var_idx] = del_q[self.indices_q[var_idx]]
                        else:
                            J[row_idx + i, len(self.indices_l) + var_idx] = 0
                    else:
                        J[row_idx + i, len(self.indices_l) + var_idx] = 0
                     
                for var_idx in range(len(self.indices_h)):
                    if row_idx == 0:
                        J[row_idx + i, len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)):
                        if  self.indices_h[var_idx] == self.indices_q[i][0]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 1
                        elif self.indices_h[var_idx] == self.indices_q[i][1]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = -1
                        else:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)):
                        J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)) + len(self.constraints[2]['fun'](x)):
                        if self.indices_h[var_idx] == list(self.source)[0]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 1
                        else:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0
                    elif row_idx == len(self.constraints[0]['fun'](x)) + len(self.constraints[1]['fun'](x)) + len(self.constraints[2]['fun'](x)) + len(self.constraints[3]['fun'](x)):
                        if self.indices_h[var_idx] == list(self.nodes)[i+1]:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = -1
                        else:
                            J[row_idx + i,len(self.indices_l) + len(self.indices_q) + var_idx] = 0 
                   
            row_idx += len(constraint_values)
        return J

    def KKT_rhs(self, x, lambda0):
        l = {self.indices_l[k]: x[k] for k in range(len(self.indices_l))}
        q = {self.indices_q[k]: x[len(self.indices_l) + k] for k in range(len(self.indices_q))}
        h = {self.indices_h[k]: x[len(self.indices_l) + len(self.indices_q) + k] for k in range(len(self.indices_h))}
        del_l = self.del_g2_del_l(l, q)
        del_q = self.del_g2_del_q(l, q)
        
        kkt_rhs = np.array([0]*(len(x)+ len(lambda0)), dtype=np.float64)

        for var_idx in range(len(self.indices_l)):
            x_index = list(self.arcs).index((self.indices_l[var_idx][0],self.indices_l[var_idx][1]))
            kkt_rhs[var_idx] = -self.C[self.indices_l[var_idx][2]] - del_l[self.indices_l[var_idx]]* lambda0[len(list(self.nodes)) + x_index] - lambda0[len(list(self.nodes)) + len(list(self.arcs)) + x_index] + 1
        for var_idx in range(len(self.indices_q)):
            x_index = len(self.nodes) + list(self.arcs).index(self.indices_q[var_idx])
            kkt_rhs[len(self.indices_l) + var_idx] = -lambda0[list(self.nodes).index(self.indices_q[var_idx][0])] + lambda0[list(self.nodes).index(self.indices_q[var_idx][1])] - del_q[self.indices_q[var_idx]]*lambda0[x_index]
        for var_idx in range(len(self.indices_h)):
            u = self.indices_h[var_idx]
            if self.indices_h[var_idx] == list(self.source)[0]:
                i = self.indices_h[var_idx]
                kkt_rhs[len(self.indices_l) + len(self.indices_q) + var_idx] = -sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((i,j))] for j in self.nodes if (i, j) in self.arcs) + sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((j,i))] for j in self.nodes if (j, i) in self.arcs) - lambda0[len(list(self.nodes)) + 2*len(list(self.arcs)) + var_idx]
            else:
                i = self.indices_h[var_idx]
                kkt_rhs[len(self.indices_l) + len(self.indices_q) + var_idx] = -sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((i,j))] for j in self.nodes if (i, j) in self.arcs) + sum(lambda0[len(list(self.nodes)) + list(self.arcs).index((j,i))] for j in self.nodes if (j, i) in self.arcs) + lambda0[len(list(self.nodes)) + 2*len(list(self.arcs)) + var_idx]

        for var_idx in range(len(self.indices_h)):
            kkt_rhs[len(x) + var_idx] = -self.constraints[0]['fun'](x)[var_idx]
        for var_idx in range(len(self.indices_q)):
            kkt_rhs[len(x) + len(self.indices_h) +  var_idx] = -self.constraints[1]['fun'](x)[var_idx]
        for var_idx in range(len(self.indices_q)):
            kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) +  var_idx] = -self.constraints[2]['fun'](x)[var_idx]
        for var_idx in range(len(self.indices_h)):
            if self.indices_h[var_idx] == list(self.source)[0]:
                kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) + len(self.indices_q) +  var_idx] = -self.constraints[3]['fun'](x)[var_idx]
            else:
                # print(self.constraints[4]['fun'](x)[var_idx])
                kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) + len(self.indices_q) +  var_idx] = -self.constraints[4]['fun'](x)[var_idx-1]   
        for var_idx in range(len(self.indices_l)):
            kkt_rhs[len(x) + len(self.indices_h) + len(self.indices_q) + len(self.indices_q) + len(self.indices_h) +  var_idx] = -self.constraints[5]['fun'](x)[var_idx]
        return kkt_rhs
        
    def solve_kkt_newton(self, x, lambda0, hessian):
        J = self.evaluate_jacobian(x)
        # print("Jacobian Matrix",J)
        H = hessian
        
        KKT_matrix = np.block([
            [H, J.T],
            [J, np.zeros((J.shape[0], J.shape[0]))]
        ])
        # print(KKT_matrix)
        # KKT_rhs = np.hstack(self.KKT_rhs(x, lambda0))
        KKT_rhs = np.hstack([self.KKT_rhs(x, lambda0).reshape(-1, 1)])  # Ensure it's a column vector

        solution = np.linalg.solve(KKT_matrix, KKT_rhs)
        delta_x = solution[:len(x)]
        delta_lambda = solution[len(x):]
        return delta_x, delta_lambda
        
    def solve_qp_problem(self, x, lambda0, hessian):
        delta_x, delta_lambda = self.solve_kkt_newton(x, lambda0, hessian)
        
        # print(f"Optimal solution: {delta_x}")
        # print(f"Lagrange multipliers: {delta_lambda}")
        return delta_x, delta_lambda
        
    def line_search(self, x, delta_x, alpha=1.0, tol=1e-4):
        new_x = x + delta_x 
        
        penalty_old = self.penalty_function(x)
        penalty_new = self.penalty_function(new_x)
        print(penalty_old)
        print(penalty_old)
        while (penalty_new).all() >= (penalty_old).all() :
            alpha *= 0.5
            new_x = x + alpha * delta_x 
            penalty_new = self.penalty_function(new_x)
        return new_x, penalty_new
    
    def penalty_function(self, x):
        objective_val = self.objective_flat(x)
        violated_constraints = sum(
            sum(np.abs(c['fun'](x)[i]) for i in range(len(c['fun'](x))) if (np.abs(c['fun'](x)[i])).all() > 0)
            for c in self.constraints
        )
        # print("objective_val", objective_val)
        # print("violated_constraints:", violated_constraints)
        return objective_val + violated_constraints

    # def penalty_function(self, x):
    #     objective_val = self.objective_flat(x)
    #     violated_constraints = sum(
    #         np.sum(np.abs(c['fun'](x)) > 1e-6)  # Count the elements that violate the threshold
    #         for c in self.constraints
    #     )
    #     print(violated_constraints)
    #     return objective_val + violated_constraints
        
    def update_hessian(self, x_old, x_new, grad_old, grad_new):
        """
        Perform the BFGS Hessian update and update the stored Hessian matrix.
        
        Parameters:
            gamma_k (np.ndarray): Difference in gradients, γ^k = grad(x^{k+1}) - grad(x^k) (n-dimensional vector).
            delta_x_k (np.ndarray): Step vector, Δx^k = x^{k+1} - x^k (n-dimensional vector).
        """
        
        gamma_k = grad_new - grad_old
        delta_x_k = x_new - x_old
        
        # Current Hessian matrix
        H_k = self.hessian
        
        # Compute the scalar terms for the update
        gamma_k_T_delta_x_k = np.dot(gamma_k, delta_x_k)  # (γ^k)^T Δx^k
        delta_x_k_T_H_k_delta_x_k = np.dot(delta_x_k.T, np.dot(H_k, delta_x_k))  # (Δx^k)^T H^k Δx^k
        
        # Check the curvature condition for numerical stability
        # if gamma_k_T_delta_x_k <= 1e-8 or delta_x_k_T_H_k_delta_x_k <= 1e-8:
        #     print("Curvature condition not satisfied. Skipping update.")
        #     return
            
        # Compute the first term: (γ^k (γ^k)^T) / (γ^k)^T Δx^k
        term1 = np.outer(gamma_k, gamma_k) / gamma_k_T_delta_x_k
        
        # Compute the second term: (H^k Δx^k (Δx^k)^T H^k) / (Δx^k)^T H^k Δx^k
        H_k_delta_x_k = np.dot(H_k, delta_x_k)  # H^k Δx^k
        term2 = np.outer(H_k_delta_x_k, H_k_delta_x_k) / delta_x_k_T_H_k_delta_x_k
        
        # Update the Hessian matrix
        hessian = H_k + term1 - term2
        return hessian
        
    def convergence_check(self, x, new_x, tol=1e-6):
        return np.linalg.norm(new_x - x) < tol
        
    def sqpSolver(self):        
        max_Q = sum(self.D[k] for k in self.nodes if k not in self.source)
        # Initial guesses for decision variables
        x0_l = np.array([0] * len(self.indices_l))
        x0_q = np.array([max_Q] * len(self.indices_q))
        x0_h = np.array([210] * len(self.indices_h))
        x = np.concatenate([x0_l, x0_q, x0_h])
                
        iteration = 0
        tolerance = 1e-6
        print(f"x{iteration}:",x)
        
        self.hessian = np.eye(len(x))
        # print("self.hessian", self.hessian)
        self.constraints = self.calculate_constraints(x)
        
        self.num_con = 0
        for constraint in self.constraints:
            c_values = constraint['fun'](x)
            self.num_con += len(c_values) 
        print("Number of constraints:",self.num_con)
        
        lambda0 = np.zeros(self.num_con)
        
        delta_x, delta_lambda = self.solve_qp_problem(x, lambda0, self.hessian)
        print(delta_x)
        x_new, penalty_new = self.line_search(x, delta_x)
        # print("x_new:", x_new)
        # print("delta_lambda:",delta_lambda)
        grad_old = self.lagrangian_gradient(x, delta_lambda)
        grad_new = self.lagrangian_gradient(x_new, delta_lambda)
        
        self.hessian = self.update_hessian(x, x_new, grad_old, grad_new)
        print(self.hessian)
        iteration += 1
        while not self.convergence_check(x, x_new, tolerance):
            x = x_new.copy()
            print(f"x{iteration}:",x_new)
            
            delta_x, delta_lambda = self.solve_qp_problem(x, delta_lambda, self.hessian)
            print(delta_x)
            x_new, penalty_new = self.line_search(x, delta_x)
            
            grad_old = self.lagrangian_gradient(x, delta_lambda)
            grad_new = self.lagrangian_gradient(x_new, delta_lambda)
            
            self.hessian = self.update_hessian(x, x_new, grad_old, grad_new)
            # print(f"Hessian{iteration}:", self.hessian)
            
            iteration += 1
            print(self.convergence_check(x, x_new, tolerance))
            if iteration > 100:
                break
        print("x optimal:",x_new)
        print("optimal solution:", self.objective_flat(x_new))
        
model_path = '../m1Basic.mod'
data_dir = '/home/nitishdumoliya/waterNetwork/data'
data_list = [
    "d1_Sample_input_cycle_twoloop",
    "d2_Sample_input_cycle_hanoi",
]

import sys
# np.set_printoptions(threshold=sys.maxsize) # 1000
# np.set_printoptions(threshold=2000) # 1000

data_number = 0
water_network = WaterNetworkModel(model_path, data_dir, data_list, data_number)
water_network.sqpSolver() 

Water Network: d1_Sample_input_cycle_twoloop 

x0: [  0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.       0.       0.
   0.       0.       0.       0.       0.       0.  

KeyboardInterrupt: 